In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import mlflow
import mlflow.tracking._model_registry.utils
from dotenv import load_dotenv

load_dotenv()

# The login and mlflow model registery are set this way because of local development. If you are running this in Databricks, you can remove the login and use the regualr mlflow registry.

mlflow.login()

mlflow.tracking._model_registry.utils._get_registry_uri_from_spark_session = (
    lambda: "databricks-uc"
)

mlflow.set_experiment("/LangChain")

In [ ]:
from mlflow.models import infer_signature

example_input = {"topic": "AI"}
example_output = {
    "content": "Why did the AI program go on a diet?\n\nBecause it wanted to lose some bytes! (get it?)",
    "additional_kwargs": {},
    "response_metadata": {
        "prompt_tokens": 17,
        "completion_tokens": 23,
        "total_tokens": 40,
    },
    "type": "ai",
    "name": None,
    "id": "run-7c12efc1-f2bb-459d-8bfd-c8bba357bf7f-0",
    "example": False,
    "tool_calls": [],
    "invalid_tool_calls": [],
    "usage_metadata": None,
}

signature = infer_signature(example_input, example_output)
signature

In [ ]:
with mlflow.start_run():
    model_info = mlflow.langchain.log_model(
        artifact_path="tech_chain",
        signature=signature,
        lc_model="LangChain.py",
        # registered_model_name="ai_recruiter.candidates.agent_prototype",
        streamable=True,
        pip_requirements="../requirements.txt",
    )

my_model = mlflow.pyfunc.load_model(model_info.model_uri)

In [ ]:
for i in my_model.predict_stream({"topic": "AI"}):
    print(i.get("content"), end="", flush=True)

In [ ]:
my_model.predict({"topic": "AI"})